# Overview

REFERENCE: https://python.langchain.com/docs/use_cases/graph/graph_cypher_qa


## Setup

In [18]:
#!pip install openai

In [19]:
#!pip install python-dotenv

In [20]:
#!pip install  langchain langchain-experimental

In [21]:
#!pip install psycopg2 python-environ

In [22]:
import os
#import openai
from dotenv import load_dotenv, find_dotenv
from langchain.llms import OpenAI
from langchain.chains import GraphCypherQAChain
from langchain.chat_models import ChatOpenAI
from langchain.graphs import Neo4jGraph
from langchain.prompts.prompt import PromptTemplate

_ = load_dotenv(find_dotenv()) 
#openai.api_key  = os.getenv('OPENAI_API_KEY')
API_KEY = os.getenv('OPENAI_API_KEY')
DBPASS=os.getenv('DBPASS')
DATABASE=os.getenv('DATABASE')
NEO4J_PASSWORD=os.getenv('NEO4J_PASSWORD')
NEO4J_URL=os.getenv('NEO4J_URL')
NEO4J_DB=os.getenv('NEO4J_DB')


graph = Neo4jGraph(
    url= NEO4J_URL, username="neo4j", password=NEO4J_PASSWORD
)

In [23]:
#print(graph.schema)

In [24]:
#graph.query("match (n:Processor{name:'grafana'})-[r:QUERIES|FORWARDS*]-(m) return n ,r,m")


In [37]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Ignore cases of the names or nouns provided in the question.

Schema:
{schema}


Cypher examples:
# How does grafana query data?
MATCH p=(:Processor)-[:QUERIES 1..6]->()
RETURN p AS path
YIELD nodes, relationships RETURN nodes, relationships
# How does data get into grafana?
match(p)-[r*3]-(q:Processor) return p,q,r
# How does Grafana get created?
MATCH p=(:Users)-[*1..3]-()-[:CREATES]->(q:Processor)
RETURN p AS path
# How does grafana get installed?
MATCH p=(:Users)-[*1..3]-()-[:CREATES]->(q:Processor)
RETURN p AS path
#Why do I not see any data in grafana?
match(p)-[*3]-(q:Processor) return p,q

Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

In [38]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True,cypher_prompt=CYPHER_GENERATION_PROMPT
    
)

### This does not add much value: return_intermediate_steps=True

In [39]:
#chaindetail = GraphCypherQAChain.from_llm(
#    ChatOpenAI(temperature=0), graph=graph, verbose=True, return_intermediate_steps=True
#)
#result = chaindetail("How does data get into grafana?")
#print(f"Intermediate steps: {result['intermediate_steps']}")
#print(f"Final answer: {result['result']}")

In [40]:
chain.run("How does data get into grafana?")



> Entering new GraphCypherQAChain chain...


ValueError: Missing some input keys: {' relationshipFilter'}

In [95]:
chain.run("What are the names of the things that Users create?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (u:Users)-[:CREATES]->(n)
RETURN n.name
Full Context:
[{'n.name': '{cluster-name}'}, {'n.name': 'AddOnDeploymentConfig-CR'}, {'n.name': '{addon-name}'}, {'n.name': 'KlusterletOperator'}, {'n.name': 'bootstrap-hub-kubeconfig'}, {'n.name': '{cluster-name}'}, {'n.name': 'ThanosObjectStoreSecret'}, {'n.name': 'ObservabilityNamespace'}, {'n.name': 'MCO-CR'}]

> Finished chain.


'The names of the things that Users create are: {cluster-name}, AddOnDeploymentConfig-CR, {addon-name}, KlusterletOperator, bootstrap-hub-kubeconfig, {cluster-name}, ThanosObjectStoreSecret, ObservabilityNamespace, and MCO-CR.'

In [96]:
chain.run("How does alert-manager get created?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH p=(:Users)-[*1..3]-()-[:CREATES]->(:Processor {name: "alert-manager"})
RETURN p AS path
Full Context:
[{'path': [{'name': 'User', 'entity': 'Users'}, 'CREATES', {'namespace': 'cluster-scoped', 'name': 'MCO-CR', 'onHub': 'true', 'type': 'Custom Resource', 'entity': 'API', 'onManaged': 'false'}, 'WATCHES', {'namespace': 'open-cluster-management', 'name': 'MulticlusterObservabilityOperator', 'onHub': 'true', 'type': 'Controller', 'entity': 'Processor', 'onManaged': 'false'}, 'CREATES', {'namespace': 'open-cluster-management-observability', 'name': 'alert-manager', 'onHub': 'true', 'type': 'Pod', 'entity': 'Processor', 'onManaged': 'false'}]}]

> Finished chain.


'The alert-manager gets created through a series of actions. First, the User creates a Custom Resource called MCO-CR in the cluster-scoped namespace. This Custom Resource is set to watch the MulticlusterObservabilityOperator, which is a Controller in the open-cluster-management namespace. Finally, the MulticlusterObservabilityOperator creates the alert-manager Pod in the open-cluster-management-observability namespace.'

In [97]:
chain.run("Why do I not see any data in grafana")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p)-[*3]-(q:Processor)
RETURN p, q
Full Context:
[{'p': {'name': 'Github Actions', 'type': 'github', 'entity': 'Processor'}, 'q': {'name': 'Github Actions', 'type': 'github', 'entity': 'Processor'}}, {'p': {'kind': 'main', 'name': 'Open Cluster Management', 'type': 'github', 'entity': 'Repository', 'group': 'policy'}, 'q': {'name': 'Github Actions', 'type': 'github', 'entity': 'Processor'}}, {'p': {'kind': 'main', 'name': 'stolostron', 'type': 'github', 'entity': 'Repository', 'group': 'policy'}, 'q': {'name': 'Github Actions', 'type': 'github', 'entity': 'Processor'}}, {'p': {'namespace': 'cluster-scoped', 'name': 'MCO-CR', 'onHub': 'true', 'type': 'Custom Resource', 'entity': 'API', 'onManaged': 'false'}, 'q': {'name': 'Github Actions', 'type': 'github', 'entity': 'Processor'}}, {'p': {'namespace': 'open-cluster-management-observability', 'name': 'ObservabilityNamespace', 'onHub': 'true', 'type': 'Namespace', 'entit

'There could be several reasons why you are not seeing any data in Grafana. It could be due to the configuration of Github Actions, Open Cluster Management, or other related components. It is also possible that there is an issue with the data sources or queries being used in Grafana. To troubleshoot this issue, you may need to check the settings and configurations of these components, ensure that the necessary data sources are properly configured, and verify that the queries being used in Grafana are correct.'

In [98]:
chain.run("How does grafana get created?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH p=(:Users)-[*1..3]-()-[:CREATES]->(q:Processor)
RETURN p AS path
Full Context:
[{'path': [{'name': 'User', 'entity': 'Users'}, 'CREATES', {'namespace': 'cluster-scoped', 'name': 'MCO-CR', 'onHub': 'true', 'type': 'Custom Resource', 'entity': 'API', 'onManaged': 'false'}, 'WATCHES', {'namespace': 'open-cluster-management', 'name': 'MulticlusterObservabilityOperator', 'onHub': 'true', 'type': 'Controller', 'entity': 'Processor', 'onManaged': 'false'}, 'CREATES', {'namespace': 'open-cluster-management-observability', 'name': 'alert-manager', 'onHub': 'true', 'type': 'Pod', 'entity': 'Processor', 'onManaged': 'false'}]}, {'path': [{'name': 'User', 'entity': 'Users'}, 'CREATES', {'namespace': 'cluster-scoped', 'name': 'MCO-CR', 'onHub': 'true', 'type': 'Custom Resource', 'entity': 'API', 'onManaged': 'false'}, 'WATCHES', {'namespace': 'open-cluster-management', 'name': 'MulticlusterObservabilityOperator', 'onHub': 'true', 

'Grafana gets created through a series of steps. First, a user creates a custom resource called MCO-CR in the cluster-scoped namespace. Then, the MulticlusterObservabilityOperator, which is a controller in the open-cluster-management namespace, watches for this custom resource. Once the MulticlusterObservabilityOperator detects the creation of the MCO-CR, it creates a pod called grafana in the open-cluster-management-observability namespace.'